## Importing libraries

In [4]:
# Import libraries
import pandas as pd
import numpy as np
import os

## Importing Data

In [17]:
#Import products data
path = r"C:\Users\ariel\OneDrive\Desktop\School work\08-2020 Instacart Basket Analysis"

In [19]:
path

'C:\\Users\\ariel\\OneDrive\\Desktop\\School work\\08-2020 Instacart Basket Analysis'

In [21]:
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)

In [59]:
#Import Orders data
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

## Finding Missing Values -Products Data

In [28]:
#Finding missing values in products data
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [30]:
#Creating new data frame to view the missing values
df_nan = df_prods[df_prods['product_name'].isnull() == True]

In [32]:
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [34]:
#Counting rows and columns of the product data
df_prods.shape

(49693, 5)

In [36]:
#Creating new dataframe without missing values
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [38]:
df_prods_clean.shape

(49677, 5)

## Finding Duplicate Values - Products Data

In [41]:
df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [43]:
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [45]:
#Checking rows and columns in clean products dataframe
df_prods_clean.shape

(49677, 5)

In [47]:
#New products dataframe with no duplicates
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [49]:
df_prods_clean_no_dups.shape

(49672, 5)

## Exporting Clean Products Dataset

In [55]:
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data','Prepared Data', 'products_checked.csv'))

## Cleaning Orders Data

## 2. 

In [66]:
#Descriptive Statistics on Orders Data
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


*First thing that I catch is an unnamed column. I'm not sure why that is there. I'm also not sure why order_id and user_id are showing up in descriptive statistics, because I changed their data types to string. Did I do something wrong in the last exercise? 

*The count in the days_since_last_order column is not the same as the rest of the columns. *The 'days_since_last_order' column has a max of 30, which seems like a long time between order placements. *The min and max for 'order_hour_of_day' seems suspicious, because those hours seem like hours where instacart would not be operating.

## 3.

In [91]:
#Checking for mixed type data
for col in df_ords.columns.tolist():
    weird = (df_ords[[col]].map(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_ords[weird]) > 0: 
        print(col)

In [95]:
# Function to check for mixed types in a column
def has_mixed_types(column):
    # Get the type of each element in the column
    types = column.apply(type)
    # Check if there is more than one unique type
    return len(types.unique()) > 1

# Example usage with your existing DataFrame 'df_ords'
# Identify and print columns with mixed types
mixed_type_columns = [col for col in df_ords.columns if has_mixed_types(df_ords[col])]

# Print the results
if mixed_type_columns:
    print("Columns with mixed types:")
    for col in mixed_type_columns:
        print(f'- {col}')
else:
    print("No columns with mixed types found.")

No columns with mixed types found.


The first function from the exercise appears to have ran successfully, but gave me no results. The function above is from ChatGPT

## 4.

No columns with mixed-types found

## 5.

In [103]:
#Finding missing values in orders data
df_ords.isnull().sum()

Unnamed: 0                    0
order_id                      0
user_id                       0
eval_set                      0
order_number                  0
orders_day_of_week            0
order_hour_of_day             0
days_since_last_order    206209
dtype: int64

*There are 206,209 missing values in the 'days_since_last_order' column. Perhaps this is because for each of those observations, there was no previous order, if it is calculating for the specific user's order. 

In [112]:
#Creating new data frame to view the missing values
df_nan2 = df_ords[df_ords['days_since_last_order'].isnull() == True]

In [114]:
df_nan2

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
0,0,2539329,1,prior,1,2,8,NaN
11,11,2168274,2,prior,1,2,11,NaN
26,26,1374495,3,prior,1,1,14,NaN
39,39,3343014,4,prior,1,6,11,NaN
45,45,2717275,5,prior,1,3,12,NaN
...,...,...,...,...,...,...,...,...
3420930,3420930,969311,206205,prior,1,4,12,NaN
3420934,3420934,3189322,206206,prior,1,3,18,NaN
3421002,3421002,2166133,206207,prior,1,6,19,NaN
3421019,3421019,2227043,206208,prior,1,1,15,NaN


In [142]:
df_ords.shape

(3421083, 8)

## 6.

In [154]:
#Imputing values using the median 
df_ords.fillna({'days_since_last_order': 7}, inplace=True)

In [152]:
#Checking for missing values again
df_ords.isnull().sum()

Unnamed: 0               0
order_id                 0
user_id                  0
eval_set                 0
order_number             0
orders_day_of_week       0
order_hour_of_day        0
days_since_last_order    0
dtype: int64

I used this method, because there was a lot of rows that were missing values in the 'days_since_last_order' column, and I am in question of the 30 day maximum, so I felt that using the median instead of the mean to impute for the missing values was the safe route.

## 7.

In [164]:
#Checking for duplicate values
df_ords_dups = df_ords[df_ords.duplicated()]

In [162]:
df_ords_dups

,Unnamed: 0,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order


## 8.

*There are no duplicate values in the orders dataframe.

If there were duplicates I would use the 'df.drop_duplicates()' method

## 9.

In [171]:
#Exporting the cleaned orders data frame
df_ords.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_checked.csv'))